### Data Augmentation

In this notebook data augmentation is performed for improving the performances of the ConvLSTM model. 

In [1]:
%cd ..
%cd ..
# move to the root directory of the git

/workspace/FLOOD_group2/models
/workspace/FLOOD_group2


In [2]:
import importlib
import torch
import copy
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch.nn as nn

# Enable interactive widgets in Jupyter Notebook
%matplotlib widget

# import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models.ConvLSTM_model.ConvLSTM_pytorch.convlstm import ConvLSTM
from models.ConvLSTM_model.ConvLSTM_pytorch.multistep_convlstm import MultiStepConvLSTM
from models.ConvLSTM_model.train_eval import train_epoch_conv_lstm, evaluation_conv_lstm
from pre_processing.encode_decode_csv import decode_from_csv
from pre_processing.normalization import * 
from pre_processing.augmentation import *
from post_processing.cool_animation import plot_animation
from post_processing.plots import *

In [3]:
# model save path
save_path = 'models/ConvLSTM_model/conv_lstm_4batch_16hidden_3kernel_augmentation.pth'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [5]:
train_val = 'train_val'
test1 = 'test1'
test2 = 'test2'
test3 = 'test3'

In [6]:
# training and validation dataset
train_dataset = decode_from_csv(train_val)

Restored inputs Shape: torch.Size([80, 1, 4, 64, 64])
Restored targets Shape: torch.Size([80, 48, 2, 64, 64])


In [7]:
transformed_dataset = augmentation(train_dataset, range_t=len(train_dataset), p_hflip=0.5, p_vflip=0.5, full=True)

The samples in the dataset before augmentation were 80
The samples in the dataset after augmentation are 160


In [8]:
# def augmentation2(train_dataset, range_t, p_hflip=0.5, p_vflip=0.5, full=True): #angles=fixed_angles, 
#     '''
#     Function for implementing data augmentation of inputs (DEM, X- and Y-Slope, 
#     Water Depth and Discharge).

#     Input: train_dataset = torch tensor, dataset with input variables
#            p_hflip, p_vflip = float, probability of horizontal and vertical flipping
#                               default = 0.5 for both
#            angles = angle degrees for dataset rotation, fixed at 0°, 90°, 180°, 270°
#     Output:  
#     '''
#     # implement transformation pipeline with horizontal and vertical flip and rotation of fixed angles
#     transformation_pipeline = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=p_hflip),
#     transforms.RandomVerticalFlip(p=p_vflip)]) 
#     #transforms.functional.rotate(train_dataset[i] for i in range(len(train_dataset)), RandomFixedRotation(angles))

#     # transform dataset
#     transformed_dataset = [transformation_pipeline(train_dataset[0]) for _ in range(range_t)]
#     print(f' Shape of transformed dataset {np.shape(transformed_dataset)}')

#     tensor_datasets = [train_dataset, transformed_dataset]

#     # Extract individual tensors and stack them along a new dimension
#     stacked_tensors = torch.stack([torch.stack((ds.tensors[0], ds.tensors[1]), dim=1) for ds in tensor_datasets], dim=0)
#     print(f'Type of stacked tensor: {type(stacked_tensors)}')
    
#     return stacked_tensors if full==True else transformed_dataset

In [9]:
# transf2 = augmentation2(train_dataset, range_t=len(train_dataset), p_hflip=0.5, p_vflip=0.5, full=True)

In [10]:
# Split dataset into train and validation
train_percnt = 0.8
train_size = int(train_percnt * len(train_dataset))
val_size = len(train_dataset) - train_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

In [11]:
# Normalize the inputs and outputs using training dataset
scaler_x, scaler_wd, scaler_q = scaler(train_set)

normalized_train_dataset = normalize_dataset(train_set, scaler_x, scaler_wd, scaler_q, train_val)
normalized_train_dataset = normalize_dataset(val_set, scaler_x, scaler_wd, scaler_q, train_val)

In [12]:
# Model
model = ConvLSTM(input_dim = normalized_train_dataset[0][0].shape[1], output_dim = normalized_train_dataset[0][1].shape[1], hidden_dim = 16, kernel_size = (3, 3),
                 num_layers = 48, batch_first=True, bias=True, return_all_layers = True).to(device)
# return all layers has to be true to obtain all the outputs I think
# num_layers refers to the number of cells and thus outputs
# Number of outputs = 4 gates * hidden_dim

In [13]:
model

ConvLSTM(
  (conv2): Conv2d(16, 2, kernel_size=(1, 1), stride=(1, 1))
  (cell_list): ModuleList(
    (0): ConvLSTMCell(
      (conv): Conv2d(20, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-47): 47 x ConvLSTMCell(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

In [14]:
# Set training parameters
learning_rate = 0.001
batch_size = 4 # Only have 64 and 16 samples for training and validation, I think should be kept small, having issues where this only works if set to 1
num_epochs = 10_000

# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Create the training and validation dataloaders to "feed" data to the model in batches
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [ ]:
train_losses = []
val_losses = []

for epoch in range(1, num_epochs+1):
    # Model training
    train_loss = train_epoch_conv_lstm(model, train_loader, optimizer, device=device)

    # Model validation
    val_loss = evaluation_conv_lstm(model, val_loader, device=device)

    if epoch == 1:
        best_loss = val_loss
    
    if val_loss<=best_loss:
        best_model = copy.deepcopy(model)
        best_loss = val_loss
        best_epoch = epoch

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch%10 == 0:
        print(f"epoch: {epoch} \t training loss: {train_loss: .2e} \t validation loss: {val_loss: .2e}")

epoch: 10 	 training loss:  4.95e-02 	 validation loss:  5.54e-02
epoch: 20 	 training loss:  4.95e-02 	 validation loss:  5.54e-02
epoch: 30 	 training loss:  4.95e-02 	 validation loss:  5.54e-02
epoch: 40 	 training loss:  4.95e-02 	 validation loss:  5.54e-02
epoch: 50 	 training loss:  4.94e-02 	 validation loss:  5.54e-02
epoch: 60 	 training loss:  4.95e-02 	 validation loss:  5.54e-02
epoch: 70 	 training loss:  4.44e-02 	 validation loss:  4.95e-02
epoch: 80 	 training loss:  3.70e-02 	 validation loss:  4.20e-02
epoch: 90 	 training loss:  3.23e-02 	 validation loss:  3.67e-02
epoch: 100 	 training loss:  2.61e-02 	 validation loss:  3.04e-02
epoch: 110 	 training loss:  2.37e-02 	 validation loss:  2.99e-02
epoch: 120 	 training loss:  2.16e-02 	 validation loss:  2.73e-02
epoch: 130 	 training loss:  2.25e-02 	 validation loss:  2.72e-02
epoch: 140 	 training loss:  2.15e-02 	 validation loss:  2.86e-02
epoch: 150 	 training loss:  1.96e-02 	 validation loss:  3.17e-02
epoc

In [ ]:
model = copy.deepcopy(best_model)
torch.save(model.state_dict(), save_path)

In [ ]:
plot_losses(train_losses, val_losses, 'ConvLSTM (+ Augmentation) ')

In [ ]:
# plot_animation(10, normalized_train_dataset, model, train_val,
#                scaler_x, scaler_wd, scaler_q, device = device, save = False)

In [ ]:
def plot_sorted(dataset, train_val, scaler_x, scaler_wd, scaler_q, model, device):
    '''
    Function for plotting the DEMs variation sorted in increasing order 
    of average loss (of Water Depth and Discharge)

    Input: dataset = tensor, normalized dataset
           train_val_test : str, Identifier of dictionary. Expects: 'train_val', 'test1', 'test2', 'test3'.
           scaler_x, scaler_wd, scaler_q = scalers for inputs (x) and targets (water depth and discharge), created 
                                            with the scaler function 
    Output: None (plot)
    '''
    
    # get inputs and outputs
    # 1st sample, 2nd input(0)/target(1), 3rd time step, 4th features, 5th/6th pixels
    
    # input = dataset[0][0]
    # target = dataset[0][1]
    
    n_samples = len(dataset)
    n_features = dataset[0][1].shape[1]
    n_pixels = dataset[0][1].shape[-1]
    time_steps = dataset[0][1].shape[0]
    
    # initialize inputs and outputs
    inputs = []
    targets = []
    
    for i in range(n_samples):
        inputs.append(dataset[i][0])
        targets.append(dataset[i][1])

    # initialize denormalization of dataset
    elevations = np.zeros((n_samples, n_pixels, n_pixels))
    water_depths = np.zeros((n_samples, time_steps, n_pixels, n_pixels))
    discharges = np.zeros((n_samples, time_steps, n_pixels, n_pixels))
    # print(discharges.shape)

    # initialize losses
    losses = torch.zeros((n_samples, n_features))
    
    for i in range(len(dataset)):
        for t in range(time_steps):
        # denormalize dataset
            elevations[i], water_depths[i], discharges[i] = denormalize_dataset(inputs[i], targets[i], train_val, 
                                                            scaler_x, scaler_wd, scaler_q)
        # make predictions
        preds = obtain_predictions(model, inputs[i], device)

        for feature in range(n_features):
            # compute MSE losses
            losses[i, feature] = nn.MSELoss()(preds[:][feature], targets[i][:][feature]) # [:, feature]
    
    # print(water_depths.shape)
    elevations_tensor = torch.tensor(elevations)

    # print(len(water_depths))
    # print(f'Shape wd: {water_depths.shape}')
    # print(f'Shape q: {discharges.shape}')

    # print(f'Size wd: {water_depths.size}')
    # print(f'Size q: {discharges.size}')

    # compute average loss for sorting dataset
    
    # loss with water depth, improve with normalized
    avg_loss = torch.mean(losses, dim=1) 
    
    # compute recall - improvement: add minimium threshold for recall (wd > 10 cm), need to denormalize targets and predictions
    # ask scaler what 10 is and plot that scaler_wd.transform(0.10) - check
    recall, _, _ = confusion_mat(dataset, model, device)

    # sorting dataset
    sorted_loss, sorted_indexes = torch.sort(avg_loss)
    print(sorted_indexes)
    # print(sorted_loss)
    # print(np.shape(sorted_loss))
    # sorted_indexes = torch.argsort(sorted_loss) #[index for index in sorted_loss]

    elevation_sorted = elevations[sorted_indexes] #[elevations[i] for i in sorted_indexes]
    # print(np.shape(elevation_sorted))
    wd_sorted, q_sorted = water_depths[sorted_indexes], discharges[sorted_indexes] #[water_depths[i] for i in sorted_indexes], [discharges[i] for i in sorted_indexes]
    sorted_recall = recall[sorted_indexes] #[recall[i] for i in sorted_indexes]
    
    # plot 
    fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    fig.subplots_adjust(wspace=0.5)

    # create second y-axis for discharge scale
    ax1_2 = axes[1].twinx()
    
    axes[0].boxplot(sorted_indexes, elevation_sorted.all()) 
    axes[1].scatter(sorted_indexes, wd_sorted[:, 0, 0, 0], color='blue', label='water depth')
    ax1_2.scatter(sorted_indexes, q_sorted[:, 0, 0, 0], color='red', label='discharge')
    axes[2].scatter(sorted_indexes, sorted_recall, color='green', label='recall')

    for ax in axes:
        ax.set_xlabel('Sample ID')
    
    axes[0].set_ylabel('Normalized variation [-]')
    axes[1].set_ylabel('Normalized Water Depth loss [-]')
    ax1_2.set_ylabel('Normalized Discharge loss [-]')
    axes[2].set_ylabel('Recall [-]')

    axes[0].set_title('Normalized DEM variation [-]')
    axes[1].set_title('Normalized MSE loss [-]')
    axes[2].set_title('Recall [-]')

    plt.legend()
    plt.show()

    return None

In [ ]:
plot_sorted(normalized_train_dataset, train_val, scaler_x, scaler_wd, scaler_q, model, device)

In [ ]:
normalized_train_dataset[0][0].shape